# Util Functions
---

- ### Formatar uma saída.

In [318]:
def get_formatted_message(poke_info):
    category_label  = 'Categories' if len(poke_info['categories']) > 1 else 'Category'
    type_label      = 'Types'      if len(poke_info['types'])      > 1 else 'Type'
    ability_label   = 'Abilities'  if len(poke_info['abilities'])  > 1 else 'Ability'
    egg_group_label = 'Egg Groups' if len(poke_info['egg_groups']) > 1 else 'Egg Group'
    

    return """\
########################################################################################
# Image: %s
# Index: %d
# Name: %s / %s (%s)
# %s: %s
# %s: %s
# ============================================================
# %s: \n# - %s
# ============================================================
# Gender Ratio: %s
# Catch Rate: %s
# ============================================================
# %s: %s
# Hatch Time: %s
# ============================================================
# Height: %s
# Weight: %s
########################################################################################
    """ % (
        poke_info['image_link'],
        poke_info['index'],
        poke_info['name'],
        poke_info['jp_name'],
        poke_info['jp_rom_name'],
        category_label,
        ' / '.join(poke_info['categories']),
        type_label,
        ' / '.join(poke_info['types']),
        ability_label,
        ' \n# - '.join(poke_info['abilities']),
        ' / '.join(poke_info['gender_ratio']) if len(poke_info['gender_ratio']) > 0 else "Genderless",
        poke_info['catch_rate'],
        egg_group_label,
        ' / '.join(poke_info['egg_groups']),
        poke_info['hatch_time'],
        ' / '.join(poke_info['height']),
        ' / '.join(poke_info['weight'])
    )


---
&nbsp;
- ### Recriar arquivos dado o nome.

In [260]:
import os

def recreate_file(file_name):
    if (os.path.exists(file_name)):
        os.remove(file_name)

    open(file_name, 'x')



# Core Functions
---

- ### Carregar a página destino e retornar o soup.

In [261]:
from urllib.request import urlopen as uOpen, Request as uReq
from bs4 import BeautifulSoup as soup

def get_poke_soup(link):
    uClient         = uReq(link, headers={'User-Agent': 'Magic Browser'})
    uCon            = uOpen(uClient)
    poke_page_html  = uCon.read()
    uCon.close()
    
    return soup(poke_page_html, 'html.parser')

---
&nbsp;
- ### Checar se é a última página a ser verificada.
    - Em alguns casos, quando tá perto do lançamento de um novo jogo e só se tem informação de um ou mais pokémons, às vezes o Bulbapédia coloca informações não oficiais com um layout diferente.

In [262]:
def check_last_page(poke_soup):
    return poke_soup.find(id='mw-content-text').table.a['href'] == "/wiki/File:BulbaShadow.png"

---
&nbsp;
- ### Pegar informações do Pokémon
    - Essa é a função core, todas as funções de extração de dados são chamadas aqui.

In [317]:
def get_poke_info(poke_soup):
    info_table = poke_soup                      \
                    .find(id='mw-content-text') \
                    .find_all('table', recursive=False)[1]

    core          = get_core_poke_info(info_table)
    types         = get_poke_types(info_table)
    abilities     = get_poke_abilities(info_table)
    gender_ratios = get_poke_gender_ratio(info_table)
    catch_rate    = get_poke_catch_ratio(info_table)
    egg_groups    = get_poke_egg_group(info_table)
    hatch_time    = get_poke_hatch_time(info_table)
    height        = get_poke_height(info_table)
    weight        = get_poke_weight(info_table)
    
    return {
        'image_link':   core[0],
        'index':        core[1],
        'name':         core[2],
        'categories':   core[3],
        'jp_name':      core[4],
        'jp_rom_name':  core[5],
        'types':        types,
        'abilities':    abilities,
        'gender_ratio': gender_ratios,
        'catch_rate':   catch_rate,
        'egg_groups':   egg_groups,
        'hatch_time':   hatch_time,
        'height':       height,
        'weight':       weight
    }

# Data Extraction Functions
---

- ### Pegar o link do próximo pokémon
    - Nessa parte da página, o Bulbapédia pode colocar trs a mais pra informar algo (geralmente relacionado ao Pokemon Go), por isso esse tratamento.

In [264]:
def get_next_pokemon_link(poke_soup):
    npl = poke_soup.find(id='mw-content-text').table
    try:
        npl = npl.findChildren('tr', recursive=False)[1]    \
                 .findChildren('td', recursive=False)[2]
    except IndexError:
        npl = npl.findChildren('tr', recursive=False)[0]    \
                 .findChildren('td', recursive=False)[2]
    finally:
        return npl.find('a')['href']            

---
&nbsp;
- ### Pegar informações core do pokémon:
    - Index (número na NationalDex)
    - Nome
    - Categoria
    - Nome (JP)
    - Nome (JP/Romaji)

In [320]:
def get_core_poke_info(info_table):
    base_info_container = info_table.tr.td.table.tr

    info_container = base_info_container.td.table.tr      \
                        .find_all('td', recursive=False)
    
    category = info_container[0].a.find_all('span')
    categories = [cat.text for cat in category]
    
    if len(categories) > 1:
        categories = list(filter(lambda cat: cat.find('Pokémon') == -1, categories))
        categories = list(map(lambda cat: cat + ' Pokémon', categories))
    
    image_link = 'https:' + info_table.img['src']
    
    return (
        image_link,                                                  # image_link
        int(base_info_container.th.find('a').text.replace('#', '')), # index
        info_container[0].big.text,                                  # name
        categories,                                                  # category
        info_container[1].span.text,                                 # jp_name
        info_container[1].i.text                                     # jp_rom_name
    )

---
&nbsp;
- ### Pegar tipos do pokémon.

In [266]:
def get_poke_types(info_table):
    types = info_table.find_all('tr', recursive=False)[1]          \
                      .table.find('td', attrs={'style': None}) \
                      .find_all('a')
    
    return [t.text for t in types if t.text != 'Unknown']


---
&nbsp;
- ### Pegar abilidades do pokémon.

In [267]:
def get_poke_abilities(info_table):
    ability_title = info_table.find('a', attrs={'title': 'Ability'})
    abilities = []

    for parent in ability_title.parents:        
        if parent.name == 'td':
            abilities_container = parent.table.find_all('td')
            
            for td in abilities_container:
                if not td.has_attr('style') or (not 'display: none' in td['style']):
                    ability = td.find('a').text

                    hidden_ability_container = td.find('small')
                    if  hidden_ability_container != None:
                        ability += f' ({hidden_ability_container.text.strip()})'

                    abilities.append(ability)
            break
    
    
    return abilities

- ### Pegar gender ratios.

In [268]:
def get_poke_gender_ratio(info_table):
    gender_ratio_titles = info_table.find('a', attrs={'title': 'List of Pokémon by gender ratio'})
    ratios = []
    
    for parent in gender_ratio_titles.parents:        
        if parent.name == 'td':
            ratios_container = parent.table.find_all('td')
            
            for td in ratios_container:
                unformatted_ratio = td.find('a')
                if not td.has_attr('style') and unformatted_ratio != None:
                    unformatted_ratio = td.find('a').text
                    ratios = list(map(lambda r: r.strip(), unformatted_ratio.split(',')))

            break
    return ratios

- ### Pegar catch rate

In [269]:
def get_poke_catch_ratio(info_table):
    catch_ratio_titles = info_table.find('a', attrs={'title': 'Catch rate'})
    ratio = ""
    
    for parent in catch_ratio_titles.parents:        
        if parent.name == 'td':
            ratio = parent.table.td.text.strip()
            break
    return ratio

- ### Pegar Egg Group.

In [270]:
def get_poke_egg_group(info_table):
    egg_group_link = info_table.find('a', attrs={'title': 'Egg Group'})
    groups = []
    
    for parent in egg_group_link.parents:
        if parent.name == 'td':
            egg_groups = parent.table.find_all('span')
            
            for egg_group in egg_groups:
                groups.append(egg_group.text)
                
            break
    
    return groups

- ### Pegar Hatch Time

In [271]:
def get_poke_hatch_time(info_table):
    hatch_time_link = info_table.find('a', attrs={'title': 'Egg cycle'})

    for parent in hatch_time_link.parents:
        if parent.name == 'td':
            hatch_time = parent.table.td.text.strip()

            hatch_time_formatted = hatch_time.split('Egg')
            if len(hatch_time_formatted) > 1:
                hatch_time = ' (Egg'.join(hatch_time_formatted)
                hatch_time += ')'
                
            return hatch_time

- ### Pegar Altura

In [272]:
def get_poke_height(info_table):
    height_link = info_table.find('a', attrs={'title': 'List of Pokémon by height'})
    heights = []
    
    for parent in height_link.parents:
        if parent.name == 'td':
            height_container = parent.table.find('tr', attrs={'style': None})
            
            for height in height_container.find_all('td'):
                heights.append(height.text.strip())
            
            break
    
    return heights

- ### Pegar Peso

In [273]:
def get_poke_weight(info_table):
    weight_link = info_table.find('a', attrs={'title': 'Weight'})
    weights = []
    
    for parent in weight_link.parents:
        if parent.name == 'td':
            weight_container = parent.table.find('tr', attrs={'style': None})
            
            for weight in weight_container.find_all('td'):
                weights.append(weight.text.strip())
            
            break
    
    return weights

# Main Cycle
---

- ### Constantes

In [346]:
import csv

BASE_URL  = 'https://bulbapedia.bulbagarden.net'
END_URL   = '/wiki/%3F%3F%3F_(Pok%C3%A9mon)'
FILE_NAME = 'pokemon_list.txt'
CSV_NAME  = 'pokemon.csv'
CURRENT_KEYS = ['image_link', 'index', 'name', 'categories', 'jp_name', 'jp_rom_name', 'types', 'abilities', 'gender_ratio', 'catch_rate', 'egg_groups', 'hatch_time', 'height', 'weight']

def get_pokemon_link(poke_name):
        return f'/wiki/{poke_name.capitalize()}_%28Pok%C3%A9mon%29'

- ### Format object to CSV

In [343]:
def format_to_csv(poke_info):
    return {
        'image_link':   f'=IMAGE(\"{poke_info["image_link"]}\")',
        'index':        poke_info['index'],
        'name':         poke_info['name'],
        'categories':   ' \n'.join(['- ' + p for p in poke_info['categories']]),
        'jp_name':      poke_info['jp_name'],
        'jp_rom_name':  poke_info['jp_rom_name'],
        'types':        ' \n'.join(['- ' + p for p in poke_info['types']]),
        'abilities':    ' \n'.join(['- ' + p for p in poke_info['abilities']]),
        'gender_ratio': ' \n'.join(['- ' + p for p in poke_info['gender_ratio']]),
        'catch_rate':   poke_info['catch_rate'],
        'egg_groups':   ' \n'.join(['- ' + p for p in poke_info['egg_groups']]),
        'hatch_time':   poke_info['hatch_time'],
        'height':       ' \n'.join(['- ' + p for p in poke_info['height']]),
        'weight':       ' \n'.join(['- ' + p for p in poke_info['weight']])
    }
        

- ### Save as CSV

In [348]:
def append_csv(poke_info):
    formatted_poke = format_to_csv(poke_info)
    keys           = formatted_poke.keys()

    with open(CSV_NAME, 'a') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=keys)
        writer.writerow(formatted_poke)
        

---
&nbsp;
- ### Main

In [349]:
recreate_file(FILE_NAME)
recreate_file(CSV_NAME)

with open(CSV_NAME, 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=CURRENT_KEYS)
    writer.writeheader()

with open(FILE_NAME, 'w') as f:
    next_pokemon_link = get_pokemon_link('Bulbasaur')

    while(next_pokemon_link != END_URL):
        poke_soup = get_poke_soup(f'{BASE_URL}{next_pokemon_link}')
        if check_last_page(poke_soup):
            break

        poke_info = get_poke_info(poke_soup)
        message   = get_formatted_message(poke_info)
        append_csv(poke_info)

        f.write(f'{next_pokemon_link}\n{message}\n')
        
        print(next_pokemon_link)
        print(message)

        next_pokemon_link = get_next_pokemon_link(poke_soup)


    print("\n\n########## FINISHED ##########\n\n")

/wiki/Bulbasaur_%28Pok%C3%A9mon%29
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/2/21/001Bulbasaur.png/250px-001Bulbasaur.png
# Index: 1
# Name: Bulbasaur / フシギダネ (Fushigidane)
# Category: Seed Pokémon
# Types: Grass / Poison
# ============================================================
# Abilities: 
# - Overgrow 
# - Chlorophyll (Hidden Ability)
# ============================================================
# Gender Ratio: 87.5% male / 12.5% female
# Catch Rate: 45 (11.9%)
# ============================================================
# Egg Groups: Monster / Grass
# Hatch Time: 5140 - 5396 steps
# ============================================================
# Height: 2'04" / 0.7 m
# Weight: 15.2 lbs. / 6.9 kg
########################################################################################
    
/wiki/Ivysaur_(Pok%C3%A9mon)
###############################################################

/wiki/Caterpie_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/5/5d/010Caterpie.png/250px-010Caterpie.png
# Index: 10
# Name: Caterpie / キャタピー (Caterpie)
# Category: Worm Pokémon
# Type: Bug
# ============================================================
# Abilities: 
# - Shield Dust 
# - Run Away (Hidden Ability)
# ============================================================
# Gender Ratio: 50% male / 50% female
# Catch Rate: 255 (43.9%)
# ============================================================
# Egg Group: Bug
# Hatch Time: 3855 - 4111 steps
# ============================================================
# Height: 1'00" / 0.3 m
# Weight: 6.4 lbs. / 2.9 kg
########################################################################################
    
/wiki/Metapod_(Pok%C3%A9mon)
########################################################################################
# Image: http

/wiki/Raticate_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/f/f4/020Raticate.png/250px-020Raticate.png
# Index: 20
# Name: Raticate / ラッタ (Ratta)
# Category: Mouse Pokémon
# Type: Normal
# ============================================================
# Abilities: 
# - Run Away (Raticate) 
# - Gluttony (Alolan Raticate) 
# - Hustle (Raticate Hidden Ability) 
# - Thick Fat (Alolan Raticate Hidden Ability)
# ============================================================
# Gender Ratio: 50% male / 50% female
# Catch Rate: 127 (26%)
# ============================================================
# Egg Group: Field
# Hatch Time: 3855 - 4111 steps
# ============================================================
# Height: 2'04" / 0.7 m
# Weight: 40.8 lbs. / 18.5 kg
########################################################################################
    
/wiki/Spearow_(Pok%C3%A9mon)
######

/wiki/Nidoran%E2%99%80_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/8/81/029Nidoran.png/250px-029Nidoran.png
# Index: 29
# Name: Nidoran♀ / ニドラン♀ (Nidoran♀)
# Category: Poison Pin Pokémon
# Type: Poison
# ============================================================
# Abilities: 
# - Poison Point 
# - Hustle (Hidden Ability)
# ============================================================
# Gender Ratio: 100% female
# Catch Rate: 235 (41.3%)
# ============================================================
# Egg Groups: Monster / Field
# Hatch Time: 5140 - 5396 steps
# ============================================================
# Height: 1'04" / 0.4 m
# Weight: 15.4 lbs. / 7.0 kg
########################################################################################
    
/wiki/Nidorina_(Pok%C3%A9mon)
###################################################################################

/wiki/Jigglypuff_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/3/3e/039Jigglypuff.png/250px-039Jigglypuff.png
# Index: 39
# Name: Jigglypuff / プリン (Purin)
# Category: Balloon Pokémon
# Types: Normal / Fairy
# ============================================================
# Abilities: 
# - Cute Charm 
# - Friend Guard (Hidden Ability)
# ============================================================
# Gender Ratio: 25% male / 75% female
# Catch Rate: 170 (32.4%)
# ============================================================
# Egg Group: Fairy
# Hatch Time: 2570 - 2826 steps
# ============================================================
# Height: 1'08" / 0.5 m
# Weight: 12.1 lbs. / 5.5 kg
########################################################################################
    
/wiki/Wigglytuff_(Pok%C3%A9mon)
###########################################################################

...

/wiki/Steelix_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/b/ba/208Steelix.png/250px-208Steelix.png
# Index: 208
# Name: Steelix / ハガネール (Haganeil)
# Category: Iron Snake Pokémon
# Types: Steel / Ground
# ============================================================
# Abilities: 
# - Rock Head (Steelix) 
# - Sheer Force (Hidden Ability) 
# - Sand Force (Mega Steelix)
# ============================================================
# Gender Ratio: 50% male / 50% female
# Catch Rate: 25 (7.7%)
# ============================================================
# Egg Group: Mineral
# Hatch Time: 6425 - 6681 steps
# ============================================================
# Height: 30'02" / 9.2 m
# Weight: 881.8 lbs. / 400.0 kg
########################################################################################
    
/wiki/Snubbull_(Pok%C3%A9mon)
#####################################

/wiki/Slugma_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/6/68/218Slugma.png/250px-218Slugma.png
# Index: 218
# Name: Slugma / マグマッグ (Magmag)
# Category: Lava Pokémon
# Type: Fire
# ============================================================
# Abilities: 
# - Magma Armor 
# - Weak Armor (Hidden Ability)
# ============================================================
# Gender Ratio: 50% male / 50% female
# Catch Rate: 190 (35.2%)
# ============================================================
# Egg Group: Amorphous
# Hatch Time: 5140 - 5396 steps
# ============================================================
# Height: 2'04" / 0.7 m
# Weight: 77.2 lbs. / 35.0 kg
########################################################################################
    
/wiki/Magcargo_(Pok%C3%A9mon)
########################################################################################
# Image: h

/wiki/Houndour_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/5/53/228Houndour.png/250px-228Houndour.png
# Index: 228
# Name: Houndour / デルビル (Delvil)
# Category: Dark Pokémon
# Types: Dark / Fire
# ============================================================
# Abilities: 
# - Early Bird 
# - Unnerve (Hidden Ability)
# ============================================================
# Gender Ratio: 50% male / 50% female
# Catch Rate: 120 (24.9%)
# ============================================================
# Egg Group: Field
# Hatch Time: 5140 - 5396 steps
# ============================================================
# Height: 2'00″ / 0.6 m
# Weight: 23.8 lbs. / 10.8 kg
########################################################################################
    
/wiki/Houndoom_(Pok%C3%A9mon)
########################################################################################
# I

/wiki/Smoochum_(Pok%C3%A9mon)
########################################################################################
# Image: https://cdn.bulbagarden.net/upload/thumb/0/0e/238Smoochum.png/250px-238Smoochum.png
# Index: 238
# Name: Smoochum / ムチュール (Muchul)
# Category: Kiss Pokémon
# Types: Ice / Psychic
# ============================================================
# Abilities: 
# - Oblivious 
# - Hydration (Hidden Ability)
# ============================================================
# Gender Ratio: 100% female
# Catch Rate: 45 (11.9%)
# ============================================================
# Egg Group: Undiscovered
# Hatch Time: 6425 - 6681 steps
# ============================================================
# Height: 1'04" / 0.4 m
# Weight: 13.2 lbs. / 6.0 kg
########################################################################################
    
/wiki/Elekid_(Pok%C3%A9mon)
########################################################################################
# Imag

KeyboardInterrupt: 

# Teste
---

In [279]:
def get_single_poke_info(poke_name, backup=False):
    next_pokemon_link = get_pokemon_link(poke_name)
    poke_soup         = get_poke_soup(f'{BASE_URL}{next_pokemon_link}')
    poke_info         = get_poke_info(poke_soup)
    
    if backup:
        with open('a.html', 'w') as f:
            f.write(poke_soup.prettify())

    print(get_formatted_message(poke_info))

In [319]:
get_single_poke_info('melmetal', True)

########################################################################################
# Image: https:////cdn.bulbagarden.net/upload/thumb/e/ea/809Melmetal.png/250px-809Melmetal.png
# Index: 809
# Name: Melmetal / メルメタル (Melmetal)
# Category: Hex Nut Pokémon
# Type: Steel
# ============================================================
# Ability: 
# - Iron Fist
# ============================================================
# Gender Ratio: Genderless
# Catch Rate: 3 (1.6%)
# ============================================================
# Egg Group: Undiscovered
# Hatch Time: 30840 - 31096 steps (Egg not obtainable)
# ============================================================
# Height: 8'02" / 2.5 m
# Weight: 1763.7 lbs. / 800.0 kg
########################################################################################
    
